In [17]:
import streamlit as st
import pandas as pd
import altair as alt
import pandas as pd
import psycopg2
from psycopg2.extensions import connection
from psycopg2.extras import RealDictCursor
from os import getenv
from dotenv import load_dotenv

In [18]:
conn = psycopg2.connect(
        host=getenv("DB_HOST"),
        user=getenv("DB_USER"),
        password=getenv("DB_PASSWORD"),
        database=getenv("DB_NAME"),
        port=getenv("DB_PORT")
    )

In [19]:
def get_cases_over_time_per_court(conn: connection):
    """
    Retrieves cases over time
    """

    query = """
            SELECT cc.court_date, c.court_name, COUNT(*) as case_count
            FROM court_case as cc
            JOIN court as c ON c.court_id = cc.court_id
            GROUP BY cc.court_date, c.court_name
            ORDER BY cc.court_date;
    """

    with conn.cursor(cursor_factory=RealDictCursor) as curs:
        curs.execute(query)
        result = curs.fetchall()
    return pd.DataFrame(result)
court_cases_over_time_df = get_cases_over_time_per_court(conn)

In [20]:
court_cases_over_time_df
for i,court in enumerate(court_cases_over_time_df['court_name']):
        if court[:10] == 'High Court':
            temp_name = court.split('(')
            court_cases_over_time_df.loc[i, 'court_name'] = 'HC (' + temp_name[1]
        elif court[:15] == 'Court of Appeal':
            temp_name = court.split('(')
            court_cases_over_time_df.loc[i, 'court_name'] = 'CoA (' + temp_name[1]


In [21]:
court_cases_over_time_df

,court_date,court_name,case_count
0,2020-01-07,HC (Technology and Construction Court),1
1,2020-01-08,HC (Chancery Division),1
2,2020-01-08,HC (Commercial Court),2
3,2020-01-08,HC (Patents Court),1
4,2020-01-09,HC (Administrative Court),1
...,...,...,...
6722,2024-08-19,Chancery Appeals,1
6723,2024-08-19,HC (Business and Property Courts),1
6724,2024-08-19,HC (Business List),1
6725,2024-08-19,HC (Circuit Commercial Court),1


In [22]:
def filtered_cases_over_time(conn:connection,filter:tuple):
    """
    Retrieves cases over time but filters by courts
    """
    query = """
    WITH original_data AS(
        SELECT cc.court_date, c.court_name, COUNT(*) as case_count
        FROM court_case as cc
        JOIN court as c ON c.court_id = cc.court_id
        GROUP BY cc.court_date, c.court_name
        ORDER BY cc.court_date)
    SELECT * FROM original_data
    WHERE court_name IN %s;
    """
    with conn.cursor(cursor_factory=RealDictCursor) as curs:
        curs.execute(query, (tuple(filter),))
        result = curs.fetchall()
    return pd.DataFrame(result)

In [23]:
df = filtered_cases_over_time(conn, ['High Court (Administrative Court)', 'Privy Council'])
df['overall_sum'] = df['case_count'].cumsum()
df

,court_date,court_name,case_count,overall_sum
0,2020-01-09,High Court (Administrative Court),1,1
1,2020-01-14,High Court (Administrative Court),2,3
2,2020-01-15,High Court (Administrative Court),1,4
3,2020-01-16,High Court (Administrative Court),3,7
4,2020-01-17,High Court (Administrative Court),1,8
...,...,...,...,...
926,2024-08-05,Privy Council,1,2080
927,2024-08-07,High Court (Administrative Court),1,2081
928,2024-08-09,High Court (Administrative Court),1,2082
929,2024-08-13,Privy Council,1,2083
